In [2]:
import pandas as pd
tables= pd.read_csv("tables_mco.csv", sep=",",encoding='utf-8',na_values="")
tables.rename(columns={'Table correspondante':'tab','Séjour / ACE':'struct', 'Public / privé': 'champ','Observation CNAMTS': 'observation' }, inplace=True)




In [3]:
tablestruct={
    'tables':tables['tab'].to_list(),
    'structure':tables['struct'].to_list(),
    'champ': tables['champ'].to_list(), 
    'observation': tables['observation'].to_list()
}

In [1]:
  pip install tableschema

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.9/68.9 KB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 KB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.1/72.1 KB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 KB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.8/114.8 KB 13.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 48.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 KB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 17.7 MB/s eta 0:00:00
  Created wheel for unicodecsv: filename=unicodecsv-0.14.1-py3-none-any.whl size=10766 sha256=9a81cc6e56d347f93b0259298afd4e3f7026ba7bdb785c440749d0b6c01c10f7
  Stored in dire

In [ ]:
df["dateCreated"][733]
res=int(df["dateCreated"][733])
res

2007

In [6]:
import re
import json
from google.colab import files
import pandas as pd 
df= pd.read_csv("MCO.csv",sep=",",encoding='utf-8',na_values="")
df.rename(columns={'Nom variable':'variables','Observation CNAMTS':'observation','Libellé':'description','Longueur':'length', 'Année de création':'dateCreated', 'Année avant disparition':'dateDeleted','Séjour / ACE':'struct','Public / privé': 'champ',  'Description table':'tab_lib'}, inplace=True)
df=df.fillna('')



##### transformation des types 
mapping_t = {
    "char": "string",
    "num":"integer",
    "date":"date",
    "Char":"string",
    "Num":"integer",
    "Date":"date"
  }
df["type"]=df["Type"].map(mapping_t)
  # Fonction qui permet de classer le type en number ou integer en fct de la taille 
def numbint(x): 
  res = re.search(",", x)
  if res:
    return "number"
  return "integer"

def strdate(x):
  if 'MOI' in x:
    return "date"
  return "string"

df["datemissing"]=''

df['nomenclature']='-'
df["rdg"]=''

  


 # Creation dy type oracle 
mapping_to = {
    "string": "string",
    "date":"date",
    "integer":"number"
    
    }

df["type_oracle"]=df["type"].map(mapping_to)

 
# Creation des bons types
for i in range(len(df['variables'])):
  if df['type'][i] in ['number', 'integer']:
    df['type'][i]=numbint(df['length'][i])  
  elif df['type'][i]=='string':
    df['type'][i]=strdate(df['variables'][i])


  

# create json file with the tables name
tableexist={
    'table':[],
    'tab_lib':[],
    'champ':[],
    'produit':'PMSI MCO'
}


for z in range(len(df['variables'])):
  if df['dateDeleted'][z] is not '':
    df['dateDeleted'][z]=df['dateDeleted'][z]+1

df['datereated']=''
for z in range(len(df['variables'])):
   # Transformation of type 
  if df['dateCreated'][z] is not '':
    df['dateCreated'][z] = int(float(df['dateCreated'][z]))
  if df['dateDeleted'][z] is not '':  
    df['dateDeleted'][z] = int (float(df['dateDeleted'][z]))
  df=df.fillna('')
  if df['Table'][z] in tableexist['table']: 
    
    new_dict={
              "name": df["variables"][z],
              "description": df["description"][z],
              "type": df["type"][z],
              "nomenclature": df["nomenclature"][z],
              "length": str(df["length"][z]),
              "format": "default",
              "dateCreated": str(df["dateCreated"][z]),
              "dateDeleted": str(df["dateDeleted"][z]),
              "dateMissing": [],
              "observation": df["observation"][z],
              "regle_gestion": df["rdg"][z],
              "type_oracle": df["type_oracle"][z]
        
          }
    
    with open('{}.json'.format(df['Table'][z]),encoding='utf-8') as f:  
      data=json.load(f)
      data.append(new_dict)
    with open('{}.json'.format(df['Table'][z]), 'w', encoding='utf-8') as f:
      json.dump(data, f, indent=4, ensure_ascii=False)
  else:
    with open('{}.json'.format(df['Table'][z]), 'w', encoding='utf-8') as f:
      json.dump([
            {"name": df["variables"][z],
              "description": df["description"][z],
              "type": df["type"][z],
              "nomenclature": df["nomenclature"][z],
              "length": str(df["length"][z]),
              "format": "default",
              "dateCreated": str(df["dateCreated"][z]),
              "dateDeleted": str(df["dateDeleted"][z]),
              "dateMissing": [],
              "observation": df["observation"][z],
              "regle_gestion": df["rdg"][z],
              "type_oracle": df["type_oracle"][z]}
        ], f, indent=4, ensure_ascii=False)
      tableexist['table'].append(df['Table'][z])
      tableexist['tab_lib'].append(df['tab_lib'][z])
      tableexist['champ'].append(df['champ'][i])
  # Mettons les fichiers sous format tableschema

from tableschema import Schema
for table in tableexist['table']:
  with open('{}.json'.format(table), "r", encoding='utf-8') as f:
    descriptor = {'fields': json.load(f)}
    schema=Schema(descriptor) 
    schema.descriptor['name']=table
    schema.descriptor['title']=tableexist['tab_lib'][tableexist['table'].index(table)]
    schema.descriptor['produit']=tableexist['produit']
    schema.descriptor['missingValues'] = ['']
            
      # La Table des sejours MCO
  for tables in tablestruct['tables']:
    if tablestruct['structure'][tablestruct['tables'].index(table)]=='Séjour' and tablestruct['champ'][tablestruct['tables'].index(table)]=="Public et Privé" : #séjours Public et Privé
      if table=='T_MCOaaB': 
          schema.descriptor['primaryKey']= [
            "ETA_NUM",
            "RSA_NUM"
            ]
          schema.descriptor['foreignKeys'] = [{"fields":["ETA_NUM"],
                                              "reference": {
                                                "resource": "T_MCOaaE",
                                                "fields":["ETA_NUM"]
                                                }
                                        
                                              }
                                            
                                            ]
      
      elif table=='T_MCOaaC':# La table de chainage patients 
        schema.descriptor['primaryKey']= [
            "ETA_NUM",
            "RSA_NUM"
            ]
        schema.descriptor['foreignKeys']=[{"fields":[
                                              "ETA_NUM",
                                              "RSA_NUM"
                                          ],
                                          "reference": {
                                                "resource": "T_MCOaaB",
                                                "fields": [
                                                    "ETA_NUM",
                                                    "RSA_NUM"
                                                ]
                                                },
                                          "description" : "Bidirectionnel"
                                          },
                                          {
                                              "fields": "NIR_ANO_17", 
                                              "reference": {
                                                "resource": "IR_BEN_R",
                                                "fields": "BEN_NIR_PSA"
                                          }
                                          },
                                          {
                                              "fields": "NIR_ANO_17", 
                                              "reference": {
                                                "resource": "IR_IMB_R",
                                                "fields": "BEN_NIR_PSA"
                                          }
                                          },
                                          {
                                              "fields": "NIR_ANO_17", 
                                              "reference": {
                                                "resource": "ER_PRS_F",
                                                "fields": "BEN_NIR_PSA"
                                          }
                                          }
                                          
                                        ]
        
      elif table=="T_MCOaaE": # Table des établissements
        schema.descriptor['primaryKey']= [
            "ETA_NUM"
            ]
      else:
        schema.descriptor['foreignKeys']=[
            {
                "fields": [
                    "ETA_NUM",
                    "RSA_NUM"

                ],
                "reference": {
                    "resource": "T_MCOaaB",
                    "fields": [
                        "ETA_NUM",
                        "RSA_NUM"
                    ]
                }
            }
        ]

    elif tablestruct['structure'][tablestruct['tables'].index(table)]=='Séjour' and tablestruct['champ'][tablestruct['tables'].index(table)]=='Public' : #séjours Public
      schema.descriptor['foreignKeys']=[{"fields":[
                                              "ETA_NUM",
                                              "RSA_NUM"
                                          ],
                                          "reference": {
                                                "resource": "T_MCOaaB",
                                                "fields": [
                                                    "ETA_NUM",
                                                    "RSA_NUM"
                                                ]
                                                }
                                          }
                                          ]
    elif tablestruct['structure'][tablestruct['tables'].index(table)]=='Séjour' and tablestruct['champ'][tablestruct['tables'].index(table)]=="Privé" : #séjours Privés: tables de facturation en établissement Privé
      if table=='T_MCOaaFA':
        schema.descriptor['primaryKey']= [
            "ETA_NUM",
            "RSA_NUM"
            ]
        schema.descriptor['foreignKeys']=[{"fields":[
                                              "ETA_NUM",
                                              "RSA_NUM"
                                          ],
                                          "reference": {
                                                "resource": "T_MCOaaC",
                                                "fields": [
                                                    "ETA_NUM",
                                                    "RSA_NUM"
                                                ]
                                                }
          }]
                                          
      else:
        schema.descriptor['foreignKeys']=[{"fields":[
                                              "ETA_NUM",
                                              "RSA_NUM"
                                          ],
                                          "reference": {
                                                "resource": "T_MCOaaFA",
                                                "fields": [
                                                    "ETA_NUM",
                                                    "RSA_NUM"
                                                ]
                                                }
                                          }
                                          ]
        # Tables des ACE 
    elif tablestruct['structure'][tablestruct['tables'].index(table)]=='ACE':
      if table=="T_MCOaaCSTC":
        schema.descriptor['primaryKey']= [
            "ETA_NUM",
            "SEQ_NUM"
            ]
        schema.descriptor['foreignKeys']=[{"fields":[
                                                          "ETA_NUM",
                                                          "SEQ_NUM"
                                                          ], 
                                              "reference": {
                                                "resource": "T_MCOaaFASTC",
                                                "fields":[
                                                          "ETA_NUM",
                                                          "SEQ_NUM"
                                                          
                                                ]},
                                          "description" : "Bidirectionnel"
                                          },
                                          {
                                              "fields": "NIR_ANO_17", 
                                              "reference": {
                                                "resource": "IR_BEN_R",
                                                "fields": "BEN_NIR_PSA"
                                          }
                                          },
                                          {
                                              "fields": "NIR_ANO_17", 
                                              "reference": {
                                                "resource": "IR_IMB_R",
                                                "fields": "BEN_NIR_PSA"
                                          }
                                          },
                                          {
                                              "fields": "NIR_ANO_17", 
                                              "reference": {
                                                "resource": "ER_PRS_F",
                                                "fields": "BEN_NIR_PSA"
                                          }
                                          }
                                          
                                        ]
      elif table=="T_MCOaaFASTC":
        schema.descriptor['primaryKey']= [
            "ETA_NUM",
            "SEQ_NUM"
            ]
      else:
        schema.descriptor['foreignKeys']=[
            {
                "fields": [
                    "ETA_NUM",
                    "SEQ_NUM"

                ],
                "reference": {
                    "resource": "T_MCOaaFASTC",
                    "fields": [
                        "ETA_NUM",
                        "SEQ_NUM"
                    ]
                }
            }
        ]
        

        
      #les autres tables : séjours MCO et ACE non individualisés, sejours et ACE à la fois

    elif tablestruct['structure'][tablestruct['tables'].index(table)] in ['Séjour (non individualisé)', 'ACE (non individualisé)','Séjour et ACE'] :
      schema.descriptor['foreignKeys']= [{"fields":["ETA_NUM"],
                                              "reference": {
                                                "resource": "T_MCOaaE",
                                                "fields":["ETA_NUM"]
                                                }
                                        
                                              }
                                            
                                            ]

          

      
    schema.descriptor['history']={"dateCreated": schema.descriptor['fields'][0]['dateCreated'],
                                          "dateDeleted": "",
                                          "dateMissing": []}
    schema.descriptor['champ']=tablestruct['champ'][tablestruct['tables'].index(table)]
    schema.descriptor['observation']=str(tablestruct['observation'][tablestruct['tables'].index(table)])                             
    schema.commit()
    schema.save('{}.json'.format(table),ensure_ascii=False)
from google.colab import files



<>:72: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:78: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:80: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:72: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:78: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:80: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<ipython-input-6-68f596f57f38>:72: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if df['dateDeleted'][z] is not '':
<ipython-input-6-68f596f57f38>:78: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if df['dateCreated'][z] is not '':
<ipython-input-6-68f596f57f38>:80: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if df['dateDeleted'][z] is not '':


In [7]:
from google.colab import files
!zip -r /content/sample_data.zip /content


  adding: content/ (stored 0%)
  adding: content/.config/ (stored 0%)
  adding: content/.config/.last_survey_prompt.yaml (stored 0%)
  adding: content/.config/.last_update_check.json (deflated 22%)
  adding: content/.config/logs/ (stored 0%)
  adding: content/.config/logs/2023.01.04/ (stored 0%)
  adding: content/.config/logs/2023.01.04/14.35.33.516796.log (deflated 55%)
  adding: content/.config/logs/2023.01.04/14.35.04.532501.log (deflated 54%)
  adding: content/.config/logs/2023.01.04/14.34.03.867433.log (deflated 92%)
  adding: content/.config/logs/2023.01.04/14.35.34.388081.log (deflated 54%)
  adding: content/.config/logs/2023.01.04/14.34.56.463059.log (deflated 86%)
  adding: content/.config/logs/2023.01.04/14.34.28.750487.log (deflated 53%)
  adding: content/.config/gce (stored 0%)
  adding: content/.config/.last_opt_in_prompt.yaml (stored 0%)
  adding: content/.config/active_config (stored 0%)
  adding: content/.config/configurations/ (stored 0%)
  adding: content/.config/conf

In [8]:
files.download('/content/sample_data.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import re
import json
from google.colab import files
import pandas as pd 
df= pd.read_csv("HAD.csv",sep=",",encoding='utf-8',na_values="")
df.rename(columns={'Nom variable':'variables','Observation CNAMTS':'observation','Libellé':'description','Longueur':'length', 'Année de création':'dateCreated', 'Année avant disparition':'dateDeleted',  'Description table':'tab_lib'}, inplace=True)
df=df.fillna('')



##### transformation des types 
mapping_t = {
    "char": "string",
    "num":"integer",
    "date":"date"
  }
df["type"]=df["Type"].map(mapping_t)
  # Fonction qui permet de classer le type en number ou integer en fct de la taille 
def numbint(x): 
  res = re.search(",", x)
  if res:
    return "number"
  return "integer"


df["datemissing"]=''

df['nomenclature']='-'
df["rdg"]=''

  


 # Creation dy type oracle 
mapping_to = {
    "string": "string",
    "date":"date",
    "integer":"number"
    
    }

df["type_oracle"]=df["type"].map(mapping_to)

 
# Creation des bons types
for i in range(len(df['variables'])):
  if df['type'][i]=='number':
    df['type'][i]=df['length'][i].apply(numbint)    

  

# create json file with the tables name
tableexist={
    'table':[],
    'tab_lib':[],
    'champ':[],
    'produit':'PMSI HAD'
}


for z in range(len(df['variables'])):
  if df['dateDeleted'][z] is not '':
    df['dateDeleted'][z]=df['dateDeleted'][z]+1

df['datereated']=''
for z in range(len(df['variables'])):
   # Transformation of type 
  if df['dateCreated'][z] is not '':
    df['dateCreated'][z] = int(float(df['dateCreated'][z]))
  if df['dateDeleted'][z] is not '':  
    df['dateDeleted'][z] = int (float(df['dateDeleted'][z]))
  df=df.fillna('')
  if df['Table'][z] in tableexist['table']: 
    
    new_dict={
              "name": df["variables"][z],
              "description": df["description"][z],
              "type": df["type"][z],
              "nomenclature": df["nomenclature"][z],
              "length": str(df["length"][z]),
              "format": "default",
              "dateCreated": str(df["dateCreated"][z]),
              "dateDeleted": str(df["dateDeleted"][z]),
              "dateMissing": [],
              "observation": df["observation"][z],
              "regle_gestion": df["rdg"][z],
              "type_oracle": df["type_oracle"][z]
        
          }
    
    with open('{}.json'.format(df['Table'][z]),encoding='utf-8') as f:  
      data=json.load(f)
      data.append(new_dict)
    with open('{}.json'.format(df['Table'][z]), 'w', encoding='utf-8') as f:
      json.dump(data, f, indent=4, ensure_ascii=False)
  else:
    with open('{}.json'.format(df['Table'][z]), 'w', encoding='utf-8') as f:
      json.dump([
            {"name": df["variables"][z],
              "description": df["description"][z],
              "type": df["type"][z],
              "nomenclature": df["nomenclature"][z],
              "length": str(df["length"][z]),
              "format": "default",
              "dateCreated": str(df["dateCreated"][z]),
              "dateDeleted": str(df["dateDeleted"][z]),
              "dateMissing": [],
              "observation": df["observation"][z],
              "regle_gestion": df["rdg"][z],
              "type_oracle": df["type_oracle"][z]}
        ], f, indent=4, ensure_ascii=False)
      tableexist['table'].append(df['Table'][z])
      tableexist['tab_lib'].append(df['tab_lib'][z])
     
  # Mettons les fichiers sous format tableschema

from tableschema import Schema
for table in tableexist['table']:
  with open('{}.json'.format(table), "r", encoding='utf-8') as f:
    descriptor = {'fields': json.load(f)}
    schema=Schema(descriptor) 
    schema.descriptor['name']=table
    schema.descriptor['title']=tableexist['tab_lib'][tableexist['table'].index(table)]
    schema.descriptor['produit']=tableexist['produit']
    schema.descriptor['missingValues'] = ['']
    if table=='T_HADaaB':
      schema.descriptor['primaryKeys']= [
        "ETA_NUM_EPMSI",
        "RHAD_NUM"
        ]
      schema.descriptor['foreignKeys'] = [{"fields":["ETA_NUM_EPMSI"],
                                           "reference": {
                                             "resource": "T_HADaaE",
                                             "fields":["ETA_NUM"]
                                             }
                                          }
                                        
                                         ]
  
    elif table=='T_HADaaC':
      schema.descriptor['primaryKeys']= [
        "ETA_NUM_EPMSI",
        "RHAD_NUM"
        ]
      schema.descriptor['foreignKeys']=[{"fields":[
                                          "ETA_NUM_EPMSI",
                                          "RHAD_NUM"
                                       ],
                                       "reference": {
                                             "resource": "T_HADaaB",
                                             "fields": [
                                                "ETA_NUM_EPMSI",
                                                "RHAD_NUM"
                                            ]
                                            },
                                       "description" : "Bidirectionnel"
                                       },
                                      {
                                          "fields": "NIR_ANO_17", 
                                          "reference": {
                                             "resource": "IR_BEN_R",
                                             "fields": "BEN_NIR_PSA"
                                      }
                                      }
                                      ]
    else:
      schema.descriptor['foreignKeys']=[{"fields":[
                                          "ETA_NUM_EPMSI",
                                          "RHAD_NUM"
                                       ],
                                       "reference": {
                                             "resource": "T_HADaaB",
                                             "fields": [
                                                "ETA_NUM_EPMSI",
                                                "RHAD_NUM"
                                            ]
                                            }
                                       }
                                      ]
    schema.descriptor['history']={"dateCreated": schema.descriptor['fields'][0]['dateCreated'],
                                    "dateDeleted": "",
                                    "dateMissing": []}
    schema.descriptor['champ']=" Public et Privé"
    schema.descriptor['observation']=""                               
    schema.commit()
    schema.save('{}.json'.format(table),ensure_ascii=False)
from google.colab import files



IndentationError: ignored

In [ ]:
import re
import json
from google.colab import files
import pandas as pd 
df= pd.read_csv("SSR.csv",sep=",",encoding='utf-8',na_values="")
df.rename(columns={'Nom variable':'variables','Observation CNAMTS':'observation','Libellé':'description','Longueur':'length', 'Année de création':'dateCreated', 'Année avant disparition':'dateDeleted','Public / privé': 'champ',  'Description table':'tab_lib'}, inplace=True)
df=df.fillna('')



##### transformation des types 
mapping_t = {
    "char": "string",
    "num":"integer",
    "date":"date"
  }
df["type"]=df["Type"].map(mapping_t)
  # Fonction qui permet de classer le type en number ou integer en fct de la taille 
def numbint(x): 
  res = re.search(",", x)
  if res:
    return "number"
  return "integer"


df["datemissing"]=''

df['nomenclature']='-'
df["rdg"]=''

  


 # Creation dy type oracle 
mapping_to = {
    "string": "string",
    "date":"date",
    "integer":"number"
    
    }

df["type_oracle"]=df["type"].map(mapping_to)

 
# Creation des bons types
for i in range(len(df['variables'])):
  if df['type'][i]=='number':
    df['type'][i]=df['length'][i].apply(numbint)    

  

# create json file with the tables name
tableexist={
    'table':[],
    'tab_lib':[],
    'champ':[],
    'produit':'PMSI SSR'
}


for z in range(len(df['variables'])):
  if df['dateDeleted'][z] is not '':
    df['dateDeleted'][z]=df['dateDeleted'][z]+1

df['datereated']=''
for z in range(len(df['variables'])):
   # Transformation of type 
  if df['dateCreated'][z] is not '':
    df['dateCreated'][z] = int(float(df['dateCreated'][z]))
  if df['dateDeleted'][z] is not '':  
    df['dateDeleted'][z] = int (float(df['dateDeleted'][z]))
  df=df.fillna('')
  if df['Table'][z] in tableexist['table']: 
    
    new_dict={
              "name": df["variables"][z],
              "description": df["description"][z],
              "type": df["type"][z],
              "nomenclature": df["nomenclature"][z],
              "length": str(df["length"][z]),
              "format": "default",
              "dateCreated": str(df["dateCreated"][z]),
              "dateDeleted": str(df["dateDeleted"][z]),
              "dateMissing": [],
              "observation": df["observation"][z],
              "regle_gestion": df["rdg"][z],
              "type_oracle": df["type_oracle"][z]
        
          }
    
    with open('{}.json'.format(df['Table'][z]),encoding='utf-8') as f:  
      data=json.load(f)
      data.append(new_dict)
    with open('{}.json'.format(df['Table'][z]), 'w', encoding='utf-8') as f:
      json.dump(data, f, indent=4, ensure_ascii=False)
  else:
    with open('{}.json'.format(df['Table'][z]), 'w', encoding='utf-8') as f:
      json.dump([
            {"name": df["variables"][z],
              "description": df["description"][z],
              "type": df["type"][z],
              "nomenclature": df["nomenclature"][z],
              "length": str(df["length"][z]),
              "format": "default",
              "dateCreated": str(df["dateCreated"][z]),
              "dateDeleted": str(df["dateDeleted"][z]),
              "dateMissing": [],
              "observation": df["observation"][z],
              "regle_gestion": df["rdg"][z],
              "type_oracle": df["type_oracle"][z]}
        ], f, indent=4, ensure_ascii=False)
      tableexist['table'].append(df['Table'][z])
      tableexist['tab_lib'].append(df['tab_lib'][z])
      tableexist['champ'].append(df['champ'][i])
  # Mettons les fichiers sous format tableschema

from tableschema import Schema
for table in tableexist['table']:
  with open('{}.json'.format(table), "r", encoding='utf-8') as f:
    descriptor = {'fields': json.load(f)}
    schema=Schema(descriptor) 
    schema.descriptor['name']=table
    schema.descriptor['title']=tableexist['tab_lib'][tableexist['table'].index(table)]
    schema.descriptor['produit']=tableexist['produit']
    schema.descriptor['missingValues'] = ['']
    if table=='T_SSRaaB':
      schema.descriptor['primaryKeys']= [
        "ETA_NUM",
        "RHA_NUM",
        "RHS_NUM"
        ]
      schema.descriptor['foreignKeys'] = [{"fields":["ETA_NUM"],
                                           "reference": {
                                             "resource": "T_SSRaaE",
                                             "fields":["ETA_NUM"]
                                             }
                                          }
                                        
                                         ]
  
    elif table=='T_SSRaaC':
      schema.descriptor['primaryKeys']= [
        "ETA_NUM",
        "RHA_NUM"
        ]
      schema.descriptor['foreignKeys']=[{"fields":[
                                          "ETA_NUM",
                                          "RHA_NUM"
                                       ],
                                       "reference": {
                                             "resource": "T_SSRaaB",
                                             "fields": [
                                                "ETA_NUM",
                                                "RHA_NUM"
                                            ]
                                            },
                                       "description" : "Bidirectionnel"
                                       },
                                      {
                                          "fields": "NIR_ANO_17", 
                                          "reference": {
                                             "resource": "IR_BEN_R",
                                             "fields": "BEN_NIR_PSA"
                                      }
                                      }
                                      ]
    else:
      schema.descriptor['foreignKeys']=[{"fields":[
                                          "ETA_NUM",
                                          "RHA_NUM",
                                          "RHS_NUM"
                                       ],
                                       "reference": {
                                             "resource": "T_SSRaaB",
                                             "fields": [
                                                "ETA_NUM",
                                                "RHA_NUM",
                                                "RHS_NUM"
                                            ]
                                            }
                                       }
                                      ]
    schema.descriptor['history']={"dateCreated": schema.descriptor['fields'][0]['dateCreated'],
                                    "dateDeleted": "",
                                    "dateMissing": []}
    schema.descriptor['champ']=tableexist['champ'][tableexist['table'].index(table)]
    schema.descriptor['observation']=""                               
    schema.commit()
    schema.save('{}.json'.format(table),ensure_ascii=False)
from google.colab import files



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
import re
import json
from google.colab import files
import pandas as pd 
df= pd.read_csv("RIP.csv",sep=",",encoding='utf-8',na_values="")
df.rename(columns={'T_RIPaaTP': 'Table','Nom variable':'variables','Observation CNAMTS':'observation','Libellé':'description','Longueur':'length', 'Année de création':'dateCreated', 'Année avant disparition':'dateDeleted','Public / privé': 'champ',  'Description table':'tab_lib'}, inplace=True)
df=df.fillna('')



##### transformation des types 
mapping_t = {
    "char": "string",
    "num":"integer",
    "date":"date"
  }
df["type"]=df["Type"].map(mapping_t)
  # Fonction qui permet de classer le type en number ou integer en fct de la taille 
def numbint(x): 
  res = re.search(",", x)
  if res:
    return "number"
  return "integer"


df["datemissing"]=''

df['nomenclature']='-'
df["rdg"]=''

  


 # Creation dy type oracle 
mapping_to = {
    "string": "string",
    "date":"date",
    "integer":"number"
    
    }

df["type_oracle"]=df["type"].map(mapping_to)

 
# Creation des bons types
for i in range(len(df['variables'])):
  if df['type'][i]=='number':
    df['type'][i]=df['length'][i].apply(numbint)    

  

# create json file with the tables name
tableexist={
    'table':[],
    'tab_lib':[],
    'champ':[],
    'produit':'PMSI RIP'
}


for z in range(len(df['variables'])):
  if df['dateDeleted'][z] is not '':
    df['dateDeleted'][z]=df['dateDeleted'][z]+1

df['datereated']=''
for z in range(len(df['variables'])):
   # Transformation of type 
  if df['dateCreated'][z] is not '':
    df['dateCreated'][z] = int(float(df['dateCreated'][z]))
  if df['dateDeleted'][z] is not '':  
    df['dateDeleted'][z] = int (float(df['dateDeleted'][z]))
  df=df.fillna('')
  if df['Table'][z] in tableexist['table']: 
    
    new_dict={
              "name": df["variables"][z],
              "description": df["description"][z],
              "type": df["type"][z],
              "nomenclature": df["nomenclature"][z],
              "length": str(df["length"][z]),
              "format": "default",
              "dateCreated": str(df["dateCreated"][z]),
              "dateDeleted": str(df["dateDeleted"][z]),
              "dateMissing": [],
              "observation": df["observation"][z],
              "regle_gestion": df["rdg"][z],
              "type_oracle": df["type_oracle"][z]
        
          }
    
    with open('{}.json'.format(df['Table'][z]),encoding='utf-8') as f:  
      data=json.load(f)
      data.append(new_dict)
    with open('{}.json'.format(df['Table'][z]), 'w', encoding='utf-8') as f:
      json.dump(data, f, indent=4, ensure_ascii=False)
  else:
    with open('{}.json'.format(df['Table'][z]), 'w', encoding='utf-8') as f:
      json.dump([
            {"name": df["variables"][z],
              "description": df["description"][z],
              "type": df["type"][z],
              "nomenclature": df["nomenclature"][z],
              "length": str(df["length"][z]),
              "format": "default",
              "dateCreated": str(df["dateCreated"][z]),
              "dateDeleted": str(df["dateDeleted"][z]),
              "dateMissing": [],
              "observation": df["observation"][z],
              "regle_gestion": df["rdg"][z],
              "type_oracle": df["type_oracle"][z]}
        ], f, indent=4, ensure_ascii=False)
      tableexist['table'].append(df['Table'][z])
      tableexist['tab_lib'].append(df['tab_lib'][z])
      tableexist['champ'].append(df['champ'][i])
  # Mettons les fichiers sous format tableschema

from tableschema import Schema
for table in tableexist['table']:
  with open('{}.json'.format(table), "r", encoding='utf-8') as f:
    descriptor = {'fields': json.load(f)}
    schema=Schema(descriptor) 
    schema.descriptor['name']=table
    schema.descriptor['title']=tableexist['tab_lib'][tableexist['table'].index(table)]
    schema.descriptor['produit']=tableexist['produit']
    schema.descriptor['missingValues'] = ['']
    if table=='T_RIPaaFB':
      schema.descriptor['primaryKeys']= [
        "ETA_NUM_EPMSI",
        "RIP_NUM"
        ]
      schema.descriptor['foreignKeys'] = [{"fields":["ETA_NUM_EPMSI"],
                                           "reference": {
                                             "resource": "T_RIPaaE",
                                             "fields":["ETA_NUM"]
                                             }
                                          }
                                        
                                         ]
  
    elif table=='T_RIPaaC':
      schema.descriptor['primaryKeys']= [
        "ETA_NUM_EPMSI",
        "RIP_NUM"
        ]
      schema.descriptor['foreignKeys']=[{"fields":[
                                          "ETA_NUM_EPMSI",
                                          "RIP_NUM"
                                       ],
                                       "reference": {
                                             "resource": "T_RIPaaFB",
                                             "fields": [
                                                "ETA_NUM_EPMSI",
                                                "RIP_NUM"
                                            ]
                                            },
                                       "description" : "Bidirectionnel"
                                       },
                                      {
                                          "fields": "NIR_ANO_17", 
                                          "reference": {
                                             "resource": "IR_BEN_R",
                                             "fields": "BEN_NIR_PSA"
                                      }
                                      }
                                      ]
    
    else:
      schema.descriptor['foreignKeys']=[{"fields":[
                                          "ETA_NUM_EPMSI",
                                          "RIP_NUM"
                                       ],
                                       "reference": {
                                             "resource": "T_RIPaaFB",
                                             "fields": [
                                                "ETA_NUM_EPMSI",
                                                "RIP_NUM"
                                            ]
                                            }
                                       }
                                      ]
    schema.descriptor['history']={"dateCreated": schema.descriptor['fields'][0]['dateCreated'],
                                    "dateDeleted": "",
                                    "dateMissing": []}
    schema.descriptor['champ']=tableexist['champ'][tableexist['table'].index(table)]
    schema.descriptor['observation']=""                               
    schema.commit()
    schema.save('{}.json'.format(table),ensure_ascii=False)
from google.colab import files



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
from tableschema import Schema
import json
for table in tableexist['tables']:
  with open('{}.json'.format(table), "r", encoding='utf-8') as f:
    descriptor = {'fields': json.load(f)}
   #descriptor="{}.json".format(table)
    schema=Schema(descriptor)
    schema.descriptor['name']=table
    schema.descriptor['title']=tableexist['tab_lib'][tableexist['table'].index(table)]
    schema.descriptor['produit']=tableexist['produit']
    schema.descriptor['missingValues'] = ['']
    if table=='NS_PRS_F': #principal table where primary key is
      schema.descriptor['primaryKeys']= [
        "CLE_DCI_JNT"
    ]
    schema.descriptor['foreignKeys'] = [{"fields":["CLE_DCI_JNT"],
                                           "reference": {
                                             "resource": "NS_PRS_F",
                                             "fields":["CLE_DCI_JNT"]
                                             }
                                          },
                                          {"fields":["BEN_IDT_ANO"],
                                           "reference": {
                                             "resource": "IR_IBA_R",
                                             "fields":["BEN_IDT_ANO"]
                                             }
                                          }
                                          
                                         ]
    schema.descriptor['history']={"dateCreated": 2016,
                                    "dateDeleted": "",
                                    "dateMissing": []}
      
      
      
    schema.commit()
    schema.save('{}.json'.format(table),ensure_ascii=False)
